In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def download_history_car(arr_urls, dest_path):
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": dest_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })

    chrome_driver_path = r'D:\GitHub\Work-Experiences\Tuan Loc Commodities\2. Scrapping Classic\2.1 My Own Scrapping Code\chromedriver_win32\chromedriver.exe'
    driver = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


    all_data = []

    for url in arr_urls:
        driver.get(url)
        time.sleep(5)  # Ensure the page has loaded

        try:
            history_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//li[@data-tab='history']"))
            )
            history_tab.click()
        except Exception:
            continue

        blocks = driver.find_elements_by_xpath("//div[@class='history-item']")

        for block in blocks:
            data = {'URL': url}

            try:
                date_element = block.find_element(By.XPATH, ".//span[contains(@class, 'typography-body2')]")
                date = date_element.text.strip()
                data['Date'] = date
            except Exception:
                pass

            third_party = ''
            try:
                third_party_element = block.find_element(By.XPATH, ".//a[contains(@class, 'underline')]")
                third_party = third_party_element.text
            except Exception:
                pass
            data['Third Party'] = third_party

            price = ''
            try:
                price_element = block.find_element(By.XPATH, ".//span[contains(@class, 'font-normal text-black')]")
                price = price_element.text
            except Exception:
                pass
            data['Price'] = price

            mileage, location = '', ''
            try:
                location_element = driver.find_element(By.XPATH, "(//*[@class='typography-body2 typography-primary-color typography-sm- typography-md- typography-lg- text-gray-600'])[1]")
                location = location_element.text
                
                mileage_element = driver.find_element(By.XPATH, "(//*[@class='typography-body2 typography-primary-color typography-sm- typography-md- typography-lg- text-gray-600'])[2]")
                mileage = mileage_element.text
            except Exception:
                pass
            data['Mileage'] = mileage
            data['Location'] = location

            event = ''
            try:
                event_element = block.find_element(By.XPATH, ".//span[contains(@class, 'typography-body2') and contains(text(), 'Sold at')]")
                event = event_element.text
            except Exception:
                try:
                    event_element = block.find_element(By.XPATH, ".//span[contains(@class, 'typography-body2') and contains(text(), 'Purchase reported')]")
                    event = event_element.text
                except Exception:
                    pass
            data['Event'] = event

            all_data.append(data)

    df = pd.DataFrame(all_data)
    df.to_csv(os.path.join(dest_path, 'output_data.csv'), index=False)
    driver.quit()

if __name__ == '__main__':
    file_path = r"D:\GitHub\Work-Experiences\Tuan Loc Commodities\2. Scrapping Classic\2.1 My Own Scrapping Code\2.1.2 Scraping Vehicle History\found links.txt"

    with open(file_path, 'r') as file:
        arr_urls = file.read().splitlines()

    dest_path = r'D:\GitHub\Work-Experiences\Tuan Loc Commodities\2. Scrapping Classic\2.0 Sample Code\data_source\car_test'

    download_history_car(arr_urls, dest_path)


C:\Users\thanh\AppData\Local\Temp\ipykernel_16024\1934161238.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 124.0.6367.119 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D8A813+48355]
	(No symbol) [0x00D1C4B1]
	(No symbol) [0x00C25358]
	(No symbol) [0x00C461AC]
	(No symbol) [0x00C41EF3]
	(No symbol) [0x00C40579]
	(No symbol) [0x00C70C55]
	(No symbol) [0x00C7093C]
	(No symbol) [0x00C6A536]
	(No symbol) [0x00C482DC]
	(No symbol) [0x00C493DD]
	GetHandleVerifier [0x00FEAABD+2539405]
	GetHandleVerifier [0x0102A78F+2800735]
	GetHandleVerifier [0x0102456C+2775612]
	GetHandleVerifier [0x00E151E0+616112]
	(No symbol) [0x00D25F8C]
	(No symbol) [0x00D22328]
	(No symbol) [0x00D2240B]
	(No symbol) [0x00D14FF7]
	BaseThreadInitThunk [0x770F7BA9+25]
	RtlInitializeExceptionChain [0x773ABE3B+107]
	RtlClearBits [0x773ABDBF+191]
